# Load the datasets

The path to dataset directory and patterns to search in those directories for the HPC, PFC recordings are loaded from the config file.

In [1]:
from phasic_tonic.detect_phasic import detect_phasic
from phasic_tonic.DatasetLoader import DatasetLoader
from phasic_tonic.helper import get_metadata
from phasic_tonic.runtime_logger import logger_setup
from phasic_tonic.utils import get_sequences

import numpy as np
import pandas as pd
import pynapple as nap
import yasa

from tqdm.auto import tqdm
from scipy.io import loadmat
from mne.filter import resample

fs_cbd = 2500
fs_os = 2500
fs_rgs = 1000

targetFs = 500
n_down_cbd = fs_cbd/targetFs
n_down_rgs = fs_rgs/targetFs
n_down_os = fs_os/targetFs

logger = logger_setup()

CONFIG_DIR = "/home/nero/phasic_tonic/data/dataset_loading.yaml"

Datasets = DatasetLoader(CONFIG_DIR)
mapped_datasets = Datasets.load_datasets()

def preprocess(signal: np.ndarray, n_down: int, target_fs=500) -> np.ndarray:
    """Downsample and remove artifacts."""
    
    logger.debug("STARTED: Resampling to 500 Hz.")
    # Downsample to 500 Hz
    data = resample(signal, down=n_down, method='fft', npad='auto')
    logger.debug("FINISHED: Resampling to 500 Hz.")
    logger.debug("Resampled: {0} -> {1}.".format(str(signal.shape), str(data.shape)))
    
    logger.debug("STARTED: Remove artifacts.")
    # Remove artifacts
    art_std, _ = yasa.art_detect(data, target_fs , window=1, method='std', threshold=4)
    art_up = yasa.hypno_upsample_to_data(art_std, 1, data, target_fs)
    data[art_up] = 0
    logger.debug("FINISHED: Remove artifacts.")
        
    data -= data.mean()
    return data

def get_start_end(hypno: np.ndarray, sleep_state_id: int):
    """Convert sleep states into lists of start and end time indices."""
    seq = get_sequences(np.where(hypno == sleep_state_id)[0])
    start = []
    end = []
    for s, e in seq:
        start.append(s)
        end.append(e)
    return (start, end)

def _detect_troughs(signal, thr):
    lidx  = np.where(signal[0:-2] > signal[1:-1])[0]
    ridx  = np.where(signal[1:-1] <= signal[2:])[0]
    thidx = np.where(signal[1:-1] < thr)[0]
    sidx = np.intersect1d(lidx, np.intersect1d(ridx, thidx))+1
    return sidx

Check the number of loaded recordings

In [33]:
cbd_cnt = 0
rgs_cnt = 0
os_cnt = 0

# Count recordings belonging to CBD dataset
for name in mapped_datasets:
    metadata = get_metadata(name)
    if metadata['treatment'] == 0 or metadata['treatment'] == 1:
        cbd_cnt += 1
    elif metadata['treatment'] == 2 or metadata['treatment'] == 3:
        rgs_cnt += 1
    elif metadata['treatment'] == 4:
        os_cnt += 1

assert cbd_cnt == 170
assert rgs_cnt == 159
assert os_cnt == 210

# Loop through the dataset

In [ ]:
with tqdm(mapped_datasets) as mapped_tqdm:
    for name in mapped_tqdm:
        metadata = get_metadata(name)
        mapped_tqdm.set_postfix_str(name)
        states_fname, hpc_fname, pfc_fname = mapped_datasets[name]
        logger.debug("Loading: {0}".format(name))

        if metadata["treatment"] == 0 or metadata["treatment"] == 1:
            n_down = n_down_cbd
        elif metadata["treatment"] == 2 or metadata["treatment"] == 3:
            n_down = n_down_rgs
        elif metadata["treatment"] == 4:
            n_down = n_down_os
        
        # Load the LFP data
        lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
        lfpPFC = loadmat(pfc_fname)['PFC'].flatten()

        # Load the states
        hypno = loadmat(states_fname)['states'].flatten()
        
        # Skip if no REM epoch is detected
        if(not (np.any(hypno == 5))):
            logger.debug("No REM detected. Skipping.")
            continue
        elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
            logger.debug("No REM longer than 10s. Skipping.")
            continue
        
        # Create Pynapple IntervalSet        
        start, end = get_start_end(hypno=hypno, sleep_state_id=5)
        rem_interval = nap.IntervalSet(start=start, end=end)
        
        # Create TsdFrame for HPC and PFC signals
        fs = n_down*targetFs
        t = np.arange(0, len(lfpHPC)/fs, 1/fs)
        lfp = nap.TsdFrame(t=t, d=np.vstack([lfpHPC, lfpPFC]).T, columns=['HPC', 'PFC'])
        
        # Detect phasic intervals
        lfpHPC_down = preprocess(lfpHPC, n_down)
        phREM = detect_phasic(lfpHPC_down, hypno, targetFs)
        
        # Create phasic REM IntervalSet
        start, end = [], []
        for rem_idx in phREM:
            for s, e in phREM[rem_idx]:
                start.append(s/targetFs)
                end.append(e/targetFs)
        phasic_interval = nap.IntervalSet(start, end)

        tonic_interval = rem_interval.set_diff(phasic_interval)
        break

In [66]:
hpc_fname = mapped_datasets['Rat5_SD8_HC_0_posttrial4'][1]
pfc_fname = mapped_datasets['Rat5_SD8_HC_0_posttrial4'][2]
hypno = mapped_datasets['Rat5_SD8_HC_0_posttrial4'][0]

# Load the LFP data
lfpHPC = loadmat(hpc_fname)['HPC'].flatten()
lfpPFC = loadmat(pfc_fname)['PFC'].flatten()

# Load the states
hypno = loadmat(hypno)['states'].flatten()

# Skip if no REM epoch is detected
if(not (np.any(hypno == 5))):
    print("No REM detected. Skipping.")
elif(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))) < 10):
    logger.debug("No REM longer than 10s. Skipping.")
    print(np.sum(np.diff(get_sequences(np.where(hypno == 5)[0]))))

# Create Pynapple IntervalSet        
start, end = get_start_end(hypno=hypno, sleep_state_id=5)
rem_interval = nap.IntervalSet(start=start, end=end)

# Create TsdFrame for HPC and PFC signals
fs = int(n_down*targetFs)
t = np.arange(0, len(lfpHPC)/fs, 1/fs)
lfp = nap.TsdFrame(t=t, d=np.vstack([lfpHPC, lfpPFC]).T, columns=['HPC', 'PFC'])

# Detect phasic intervals
lfpHPC_down = preprocess(lfpHPC, n_down)
phREM = detect_phasic(lfpHPC_down, hypno, targetFs)

/tmp/ipykernel_23139/1249661457.py:21: UserWarning: Some epochs have no duration
  rem_interval = nap.IntervalSet(start=start, end=end)


## Access the HPC and PFC signals during phasic REM

In [ ]:
lfp.restrict(phasic_interval)

In [5]:
phrem_hpc = [lfp["HPC"].restrict(phasic_interval[i]).to_numpy() for i in range(len(phasic_interval))]
tonic_hpc = [lfp["HPC"].restrict(tonic_interval[i]).to_numpy() for i in range(len(tonic_interval))]

In [25]:
np.savez(name+'_phasic', *phrem_hpc)
np.savez(name+'_tonic', *tonic_hpc)

In [ ]:
data = np.load('Rat5_SD8_HC_0_posttrial3_phasic.npz')
phrem = [data[key] for key in data]
phrem

## Access HPC and PFC signals during tonic REM

In [ ]:
lfp.restrict(tonic_interval)